# Introdução

### O que está acontecendo nos comandos abaixo?

No R, existe uma estrutura de dados, conhecida como vetor. Sempre que você desejar armazenar dados em uma estrutura única, você pode criar um vetor, inserindo itens da seguinte forma:

c ( valor $x_1$, valor $x_2$, ... , valor $x_n$).

Estes valores podem ser números, strings (mas devem ser do mesmo tipo).

Ao se manipular dados no R, é essencial que se entenda o funcionamento de operações em vetores, pois os dataframes (principal estrutura utilizada na análise de dados) são um conjunto de vetores associado a um índice. A convenção utilizada na análise de dados é que cada linha represente um conjunto de variáveis (isto é, por exemplo, a primeira linha poderia se referir a um paciente em um hospital, e cada coluna da primeira linha deverá se referir às informações atribuídas a ele, como por exemplo, a coluna 1 ser a altura, a coluna 2 o peso, a coluna 3 a pressão, etc).

No R, também existem as listas, que são estruturas cuja ideia é parecida com a dos vetores, porém podem armazenar diversos tipos diferentes de variáveis (strings, fatores, números). É muito útil para consolidar dataframes. Açém disto, duas operações que são importantes para a construção dos mesmos, são o **rbind** e **merge**. O primeiro é uma abreviação de "row bind", e serve para se acrescentar mais observações. Já o último se refere ao cruzamento de dataframes distintos (i.e., concatenar dois df's em um - operações comuns são *left join* e *inner join* por exemplo). 

No código abaixo, são criados três vetores distintos, mas cujo intuito seria representar um dataframe. Suponha que trate-se de um experimento em que há dois grupos de voluntários (cada um representado por um número - id): aqueles que passaram pelo tratamento A, e aqueles que passaram pelo B (trat). Assim, realizado o experimento, cada um teve suas medidas tiradas (representadas por $y$).

In [1]:
id = 1:20
y<- c(89.7, 81.4, 84.5, 84.8, 87.3, 79.7, 85.1, 81.7, 83.7, 84.5, 84.7, 86.1, 83.2, 91.9, 86.3, 79.3, 82.6, 89.1, 83.7, 88.5)
trat <- c ( rep ( "A" , 10 ) ,rep ( "B", 10 ) )
# Juntando os diferentes vetores em uma lista
juntos = list(id,trat,y)

In [2]:
# Transformação a lista em um dataframe
df = as.data.frame ( juntos , stringsAsFactors = F )
# assimilando nomes as colunas
colnames(df) = c('ID','Trat','Resposta')
# mostrando primeiras 5 linhas do df
head ( df , 5 )

,ID,Trat,Resposta
,<int>,<chr>,<dbl>
1,1,A,89.7
2,2,A,81.4
3,3,A,84.5
4,4,A,84.8
5,5,A,87.3


In [3]:
# Suponha que desejamos acrescentar dois pacientes
# O primeiro tem informações 90 (y) "B" (method) - id 21
# O segundo tem informações 84 (y) "A" (method) - id 22
dfExpandido = rbind ( df , list ( 21 , "B" , 90.0 ) , list ( 22 , "A" , 84.0 ) )
# mostrando ultimas 5 linhas do df
tail(dfExpandido , 5)

,ID,Trat,Resposta
,<dbl>,<chr>,<dbl>
18,18,B,89.1
19,19,B,83.7
20,20,B,88.5
21,21,B,90.0
22,22,A,84.0


In [4]:
# Suponha agora que temos a informação da altura de parte dos pacientes
# Sejam 15 pacientes
n_ = 15
# acrescentando uma semente para processos aleatorios
set.seed(9297791)
# Selecionados aleatoriamente, sem reposicao 15 pacientes
id = sample(x = 1:22 , size = n_ , replace = F )
# gerando valores aleatorios
y2 = rnorm(n = n_ , mean = 1.7 , sd = .1 )
# arredondando para apenas duas casas decimais
y2 = round( x = y2 , digits = 2 )
# criando df2
# Repare que pode-se dar nomes aos itens de uma lista
df2 = as.data.frame(list( Paciente = id , Altura = y2 ))
# primeiras 5 linhas
head(df2 , 5)

,Paciente,Altura
,<int>,<dbl>
1,1,1.76
2,16,1.65
3,21,1.73
4,8,1.70
5,3,1.75


In [5]:
# E que desejamos saber a altura dos voluntarios
# Neste caso, sera necessario fazer um merge entre ambos os df's

dfMerge = merge( x = dfExpandido , y = df2 , by.x = 'ID', by.y = 'Paciente' , all.x = T , all.y = F )
head(dfMerge,10)

,ID,Trat,Resposta,Altura
,<dbl>,<chr>,<dbl>,<dbl>
1,1,A,89.7,1.76
2,2,A,81.4,1.79
3,3,A,84.5,1.75
4,4,A,84.8,1.77
5,5,A,87.3,1.65
6,6,A,79.7,1.71
7,7,A,85.1,1.78
8,8,A,81.7,1.70
9,9,A,83.7,NA


In [6]:
# Suponha que desejamos saber quais voluntarios tem a medida y superior a 85
# pode-se criar ou acessar colunas do df utilizando o operador $
# a funcao ifelse retorna os valores do segundo argumento se o primeiro argumento for verdadeiro
# caso contrario, retorna o terceiro argumento
dfMerge$Indica = ifelse ( test = dfMerge$Resposta >= 85 , yes = 1 , no = 0 )
head ( dfMerge , 5 )

,ID,Trat,Resposta,Altura,Indica
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,1,A,89.7,1.76,1
2,2,A,81.4,1.79,0
3,3,A,84.5,1.75,0
4,4,A,84.8,1.77,0
5,5,A,87.3,1.65,1


In [7]:
# Quantas informações estão acima de 85?

table ( dfMerge$Indica )


 0  1 
13  9 

In [8]:
# Quantas informações estão acima de 85 por tratamento?

table ( dfMerge$Trat , dfMerge$Indica )

   
    0 1
  A 8 3
  B 5 6

In [9]:
# Freq rel por linha?

prop.table ( table ( dfMerge$Trat , dfMerge$Indica ) , 1 )

   
            0         1
  A 0.7272727 0.2727273
  B 0.4545455 0.5454545

In [10]:
# Freq rel por coluna?

prop.table ( table ( dfMerge$Trat , dfMerge$Indica ) , 2 )

   
            0         1
  A 0.6153846 0.3333333
  B 0.3846154 0.6666667

In [11]:
# Freq rel total?

prop.table ( table ( dfMerge$Trat , dfMerge$Indica ) )

   
            0         1
  A 0.3636364 0.1363636
  B 0.2272727 0.2727273

In [12]:
# Outra forma que poderíamos ter construído "Indica"

for ( i in 1 : nrow ( dfMerge ) ){
    
    if ( dfMerge [ i , "Resposta" ] >= 85 ){
        
        dfMerge [ i , "Indica2" ] = 1
        
    } else {
        
        dfMerge [ i , "Indica2" ] = 0
        
    }
    
    
}

head ( dfMerge , 5 )

# Esta é uma forma muito mais lenta de se fazer a criação da variável

,ID,Trat,Resposta,Altura,Indica,Indica2
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,A,89.7,1.76,1,1
2,2,A,81.4,1.79,0,0
3,3,A,84.5,1.75,0,0
4,4,A,84.8,1.77,0,0
5,5,A,87.3,1.65,1,1


In [13]:
# Suponha que iremos fazer um experimento com estes voluntarios, e lançaremos um dado de 4 faces
# honesto para decidir qual das 4 possibilidades
# Podemos definir da seguinte forma
# runif - gera um valor da distribuicao uniforme [0,1]

# Garantir reproducibilidade
set.seed ( 9297791 )
dfMerge$resultadosDado = runif ( nrow ( dfMerge ) )
dfMerge$TratamentoNovo = ifelse ( dfMerge$resultadosDado < .25 , "A" ,
                                    ifelse ( dfMerge$resultadosDado < .5 , "B" , 
                                           ifelse ( dfMerge$resultadosDado < .75 , "C" , "D" )))

head ( dfMerge , 5 )

,ID,Trat,Resposta,Altura,Indica,Indica2,resultadosDado,TratamentoNovo
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,A,89.7,1.76,1,1,0.4653364,B
2,2,A,81.4,1.79,0,0,0.8386083,D
3,3,A,84.5,1.75,0,0,0.6845161,C
4,4,A,84.8,1.77,0,0,0.5291512,C
5,5,A,87.3,1.65,1,1,0.1756871,A


In [14]:
# Suponha que se deseje apagar a coluna "resultadosDado"
dfMerge$resultadosDado = NULL
head ( dfMerge , 5 )

,ID,Trat,Resposta,Altura,Indica,Indica2,TratamentoNovo
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,A,89.7,1.76,1,1,B
2,2,A,81.4,1.79,0,0,D
3,3,A,84.5,1.75,0,0,C
4,4,A,84.8,1.77,0,0,C
5,5,A,87.3,1.65,1,1,A


## Próximos comandos

In [15]:
tapply(y, method, mean) #médias dos grupos

A     B 
84.24 85.54

In [17]:
tapply(y, method, sd)   #desvio padrão dos grupos

A        B 
2.901800 3.650327

In [18]:
sd(y[1:10])/sqrt(10)  #erro padrão de “A”

[1] 0.9176298

In [19]:
sd(y[11:20])/sqrt(10) #erro padrão de “B”

[1] 1.154335

A ideia destes blocos é calcular a média e os desvio e erro padrões destes dois tratamentos.
Como podemos fazer isso em nosso data frame?

In [47]:
# uma forma possivel

tratamentos = unique ( dfExpandido$method )

for ( trat in tratamentos ){
    
    media = mean ( dfExpandido[ dfExpandido$method == trat , "y" ] )
    dp = sd ( dfExpandido[ dfExpandido$method == trat , "y" ] )
    ep = dp / sqrt ( nrow ( dfExpandido[ dfExpandido$method == trat , ] ) )
    
    print ( paste0 ( "Tratamento " , trat ) )
    print ( paste0 ( "A média é: " , media ) )
    print ( paste0 ( "O desvio padrão é: " , dp ) )
    print ( paste0 ( "O erro padrão é: " , ep ) )
    
}

[1] "Tratamento A"
[1] "A média é: 84.2181818181818"
[1] "O desvio padrão é: 2.75384029377819"
[1] "O erro padrão é: 0.830314089729486"
[1] "Tratamento B"
[1] "A média é: 85.9454545454545"
[1] "O desvio padrão é: 3.71493301053024"
[1] "O erro padrão é: 1.12009444702129"


In [49]:
# Outra forma

tapply(dfExpandido$y, dfExpandido$method, mean)
tapply(y, method, sd)

A        B 
84.21818 85.94545

A        B 
2.901800 3.650327

In [50]:
sd( dfExpandido [ dfExpandido$method == "A" , ]$y )/sqrt( nrow ( dfExpandido [ dfExpandido$method == "A" , ] ) )  #erro padrão de “A”

[1] 0.8303141

In [51]:
sd( dfExpandido [ dfExpandido$method == "B" , ]$y )/sqrt( nrow ( dfExpandido [ dfExpandido$method == "A" , ] ) )  #erro padrão de “A”

[1] 1.120094

In [33]:
suppressMessages ( library ( dplyr ) )
options( dplyr.summarise.inform = FALSE )

In [45]:
# Outra forma

infos = dfExpandido %>% group_by ( method ) %>% summarise ( media = mean ( y ) , dp = sd ( y ) , n = n ( ) )
infos

method,media,dp,n
<fct>,<dbl>,<dbl>,<int>
A,84.21818,2.753840,11
B,85.94545,3.714933,11


In [40]:
infos %>% filter ( method == "A" )

method,media,dp,ep
<fct>,<dbl>,<dbl>,<dbl>
A,84.21818,2.75384,0.8303141


In [42]:
infos %>% mutate ( ep = dp / sqrt ( n ) )

method,media,dp,n,ep
<fct>,<dbl>,<dbl>,<int>,<dbl>
A,84.21818,2.753840,11,0.8303141
B,85.94545,3.714933,11,1.1200944
